<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/Development/District-Builtup-Index-GHSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import ee
import geemap
import geopandas as gpd
import pandas as pd

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

In [4]:
image_collection = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")

[ee.ImageCollection.aggregate_array](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-aggregate_array):  Aggregates over a given property of the objects in a collection, calculating a list of all the values of the selected property.

In [19]:
years = image_collection.aggregate_array('system:index').getInfo()
print(years)

['1975', '1980', '1985', '1990', '1995', '2000', '2005', '2010', '2015', '2020', '2025', '2030']


In [66]:
# Load the district boundaries dataset
districts = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM0_NAME', 'India'))
districts = districts.select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area'])
feature = districts.first()

In [72]:
%%time

# Load the GHSL built-up surface dataset
ghsl_built_up = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")

# Function to calculate the built-up surface area for each district for a given year
def calculate_built_up_area(feature, year):
    built_up_image = ghsl_built_up.filterDate(f'{year}-01-01', f'{year}-12-31').first()

    built_up_image = built_up_image

    built_up_area = built_up_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=100,  # Scale to match GHSL data resolution
        maxPixels=1e9
    ).get('built_surface')

    return feature.set({'year': year, 'area_m2': built_up_area})

# Initialize an empty list to store results
results_list = []

# Loop through each year and calculate the built-up area for each district
for year in years:
    districts_with_built_up_area = districts.map(lambda feature: calculate_built_up_area(feature, year))
    gdf = geemap.ee_to_gdf(districts_with_built_up_area)
    results_list.append(gdf)

CPU times: user 34.7 s, sys: 2.15 s, total: 36.9 s
Wall time: 3min 31s


In [73]:
# Convert the results list to a pandas dataframe
df = pd.concat(results_list)
gdf = gpd.GeoDataFrame(df)

In [74]:
gdf.head()

,geometry,ADM1_NAME,ADM2_CODE,ADM2_NAME,Shape_Area,area_m2,year
0,"POLYGON ((76.48194 29.51891, 76.48206 29.51872...",Haryana,17660,Karnal,0.240933,1.724510e+07,1975
1,"POLYGON ((76.4125 30.04788, 76.41295 30.046, 7...",Haryana,17661,Kurukshetra,0.171739,1.194741e+07,1975
2,"POLYGON ((75.889 28.39892, 75.88923 28.39366, ...",Haryana,17662,Mahendragarh,0.177959,5.877115e+06,1975
3,"POLYGON ((76.46556 29.24227, 76.47164 29.23409...",Haryana,17665,Sonepat,0.205076,1.850835e+07,1975
4,"POLYGON ((76.16629 29.92866, 76.17149 29.92323...",Haryana,70134,Kaithal,0.214695,1.140200e+07,1975
